In [16]:
import numpy as np
from qiskit import QuantumCircuit, qasm2
from qiskit.circuit.library import QFT

def generate_full_evolution_circuit(initial_state_qasm_path, t, n_x, n_y):
    """
    生成完整的时间演化电路（初态制备 + 时间演化）
    
    参数:
        initial_state_qasm_path: 初态制备的QASM文件路径
        t: 演化时间
        n_x: x方向量子比特数
        n_y: y方向量子比特数
        
    返回:
        完整量子电路 (QuantumCircuit)
    """
    n_total = n_x + n_y
    
    # 从QASM文件创建初态电路
    initial_circuit = QuantumCircuit.from_qasm_file(initial_state_qasm_path)
    
    # 验证量子比特数
    if initial_circuit.num_qubits != n_total:
        raise ValueError(f"初态电路的量子比特数({initial_circuit.num_qubits})与系统大小({n_total})不匹配")
    
    # 创建完整电路
    full_circuit = QuantumCircuit(n_total)
    
    # 添加初态制备电路（完整保留原结构）
    full_circuit.compose(initial_circuit, inplace=True)
    
    # 添加屏障标记
    # full_circuit.barrier()
    
    # 核心演化结构
    def add_direction_evolution(qc, qubits, t, direction):
        """添加优化后的单方向演化电路"""
        n_qubits = len(qubits)
        
        # 应用QFT
        qft = QFT(num_qubits=n_qubits, do_swaps=True, inverse=False, name=f"QFT_{direction}")
        qc.append(qft, qubits)
        
        # 添加相位旋转（角度与量子比特位置相关）
        for i, q in enumerate(qubits):
            # 计算角度：指数递减 2^(n-1), 2^(n-2), ..., 2^0
            exponent = n_qubits - i - 1
            angle = -t * (2**(2 * exponent)) * np.pi
            qc.rz(angle, q)
        
        # 应用逆QFT
        iqft = QFT(num_qubits=n_qubits, do_swaps=True, inverse=True, name=f"IQFT_{direction}")
        qc.append(iqft, qubits)
    
    # 为x方向添加演化
    x_qubits = list(range(n_x))
    add_direction_evolution(full_circuit, x_qubits, t, 'x')
    
    # 为y方向添加演化
    y_qubits = list(range(n_x, n_x + n_y))
    add_direction_evolution(full_circuit, y_qubits, t, 'y')
    
    return full_circuit

def generate_full_qasm(initial_state_qasm_path, t, n_x, n_y, output_file="full_evolution.qasm"):
    """
    生成完整电路的QASM（初态制备 + 时间演化）
    
    参数:
        initial_state_qasm_path: 初态制备的QASM文件路径
        t: 演化时间
        n_x: x方向量子比特数
        n_y: y方向量子比特数
        output_file: 输出文件名
        
    返回:
        QASM字符串
    """
    # 生成完整电路
    full_circuit = generate_full_evolution_circuit(initial_state_qasm_path, t, n_x, n_y)
    
    # 直接生成QASM字符串
    qasm_str = qasm2.dumps(full_circuit)
    
    # 保存文件
    with open(output_file, 'w') as f:
        f.write(qasm_str)
    
    return qasm_str, full_circuit





In [17]:
    # 参数设置
    initial_qasm_file = "diverging_flow_t=0.qasm"  # 替换为您的QASM文件路径
    output_qasm_file = "diverging_flow_t=pi_over_2_test_2.qasm"
    t_value = np.pi / 2  # 演化时间
    n_x, n_y = 5, 5  # 系统尺寸
    
    # 生成完整QASM
    qasm_output, full_circuit = generate_full_qasm(
        initial_state_qasm_path=initial_qasm_file,
        t=t_value,
        n_x=n_x,
        n_y=n_y,
        output_file=output_qasm_file
    )
    
    print(f"已生成完整电路QASM，保存至{output_qasm_file}")
    print(f"总量子比特数: {n_x + n_y}")
    print(f"电路深度: {full_circuit.depth()}")
    print(f"门计数: {full_circuit.count_ops()}")

已生成完整电路QASM，保存至diverging_flow_t=pi_over_2_test_2.qasm
总量子比特数: 10
电路深度: 16
门计数: OrderedDict([('u', 27), ('rz', 10), ('cz', 9), ('QFT_x', 1), ('IQFT_x', 1), ('QFT_y', 1), ('IQFT_y', 1)])
